In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.naive_bayes import GaussianNB
from time import process_time
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import RFE
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import cross_val_score, cross_val_predict, KFold

# <font color=black>Random Forest Feature Selection and Classifiers</font>

In [2]:
df = pd.read_csv('dataset_ros.csv')
dataset = df.copy()
dataset.head()

,Destination_Port,Flow_Duration,Total_Fwd_Packets,Total_Backward_Packets,Total_Length_of_Fwd_Packets,Total_Length_of_Bwd_Packets,Fwd_Packet_Length_Max,Fwd_Packet_Length_Min,Fwd_Packet_Length_Mean,Fwd_Packet_Length_Std,...,min_seg_size_forward,Active_Mean,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min,Label_Category_Num
0,0.000809,1.475758e-03,0.000009,0.000007,5.426357e-06,1.556176e-07,0.001410,0.015054,0.005891,0.0,...,3.725292e-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.000809,1.980333e-04,0.000005,0.000003,3.953488e-06,1.266300e-07,0.002055,0.021935,0.008585,0.0,...,5.960467e-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.000809,1.633333e-06,0.000009,0.000007,4.651163e-06,3.264917e-07,0.001209,0.012903,0.005050,0.0,...,3.725292e-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.000809,1.208725e-03,0.000005,0.000003,3.255814e-06,1.540919e-07,0.001692,0.018065,0.007070,0.0,...,3.725292e-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.861494,2.500000e-08,0.000009,0.000000,9.302326e-07,0.000000e+00,0.000242,0.002581,0.001010,0.0,...,3.725292e-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [3]:
df_x = dataset.iloc[:,:-1]
df_y = dataset.iloc[:,[-1]]

In [4]:
t1_start = process_time() 
sel = RFE(RandomForestClassifier(n_estimators = 10, random_state = 10), n_features_to_select = 27)
sel.fit(df_x, df_y)
t1_stop = process_time()
print("time:", t1_stop-t1_start)

C:\Users\deneysel\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


time: 17510.328125


In [5]:
sel.get_support()

array([ True, False,  True,  True,  True, False,  True, False,  True,
        True,  True, False,  True,  True, False,  True, False, False,
       False, False, False, False, False,  True,  True, False, False,
       False, False, False, False, False, False, False, False, False,
        True, False, False,  True,  True,  True, False, False, False,
       False,  True, False, False, False, False, False,  True,  True,
        True, False, False, False, False, False, False,  True,  True,
        True, False,  True,  True,  True, False, False, False, False,
       False, False, False, False, False])

In [6]:
selected_feat= df_x.columns[(sel.get_support())]
len(selected_feat)

27

In [7]:
print(selected_feat)

Index(['Destination_Port', 'Total_Fwd_Packets', 'Total_Backward_Packets',
       'Total_Length_of_Fwd_Packets', 'Fwd_Packet_Length_Max',
       'Fwd_Packet_Length_Mean', 'Fwd_Packet_Length_Std',
       'Bwd_Packet_Length_Max', 'Bwd_Packet_Length_Mean',
       'Bwd_Packet_Length_Std', 'Flow_Packets_s', 'Fwd_IAT_Max', 'Fwd_IAT_Min',
       'Fwd_Packets_s', 'Max_Packet_Length', 'Packet_Length_Mean',
       'Packet_Length_Std', 'PSH_Flag_Count', 'Average_Packet_Size',
       'Avg_Fwd_Segment_Size', 'Avg_Bwd_Segment_Size', 'Subflow_Fwd_Packets',
       'Subflow_Fwd_Bytes', 'Subflow_Bwd_Packets', 'Init_Win_bytes_forward',
       'Init_Win_bytes_backward', 'act_data_pkt_fwd'],
      dtype='object')


In [8]:
df_x2 = df_x[selected_feat]

In [9]:
res_df = pd.concat([df_x2, df_y], axis = 1)

In [10]:
res_x = res_df.iloc[:,:-1]
res_y = res_df.iloc[:,[-1]]

In [11]:
model = RandomForestClassifier()
#model = DecisionTreeClassifier()
#model = GaussianNB()
#model = LogisticRegression()

In [12]:
t1_start = process_time() 
kfold = KFold(n_splits = 10, random_state = 1, shuffle = True)
results = cross_val_score(model, res_x, res_y, scoring = 'accuracy', cv = kfold)
predict = cross_val_predict(model, res_x , res_y,cv = 10)
print("Accuracy : ", results.mean())
print("Classification Report : \n", classification_report(predict, res_y))
print("Confusion Metrix : \n", confusion_matrix(predict, res_y))
t1_stop = process_time()
print("time:", t1_stop-t1_start)

C:\Users\deneysel\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\deneysel\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\deneysel\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\deneysel\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using rav

C:\Users\deneysel\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\deneysel\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:876: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\deneysel\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\deneysel\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:876: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using rav

Accuracy :  0.9993775251166713
Classification Report : 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00   2014157
           1       1.00      1.00      1.00    908062
           2       1.00      1.00      1.00    565374
           3       1.00      1.00      1.00    463717
           4       1.00      1.00      1.00     50405
           5       0.99      0.93      0.96      6430
           6       1.00      1.00      1.00      8081

    accuracy                           1.00   4016226
   macro avg       1.00      0.99      0.99   4016226
weighted avg       1.00      1.00      1.00   4016226

Confusion Metrix : 
 [[2013919      73      72      20       0      65       8]
 [    867  907188       4       2       0       0       1]
 [    930       2  564442       0       0       0       0]
 [     11       7       0  463699       0       0       0]
 [      1       0       0       0   50404       0       0]
 [    447       0       0     

In [13]:
model = DecisionTreeClassifier()
t1_start = process_time() 
kfold = KFold(n_splits = 10, random_state = 1, shuffle = True)
results = cross_val_score(model, res_x, res_y, scoring = 'accuracy', cv = kfold)
predict = cross_val_predict(model, res_x , res_y,cv = 10)
print("Accuracy : ", results.mean())
print("Classification Report : \n", classification_report(predict, res_y))
print("Confusion Metrix : \n", confusion_matrix(predict, res_y))
t1_stop = process_time()
print("time:", t1_stop-t1_start)

Accuracy :  0.9994362867163795
Classification Report : 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00   2014648
           1       1.00      1.00      1.00    907633
           2       1.00      1.00      1.00    565294
           3       1.00      1.00      1.00    463747
           4       1.00      1.00      1.00     50411
           5       0.99      0.93      0.96      6414
           6       1.00      1.00      1.00      8079

    accuracy                           1.00   4016226
   macro avg       1.00      0.99      0.99   4016226
weighted avg       1.00      1.00      1.00   4016226

Confusion Metrix : 
 [[2014316      67     175       6       0      84       0]
 [    423  907205       0       2       0       0       3]
 [    937       1  564355       0       0       0       1]
 [     27       7       0  463713       0       0       0]
 [      7       0       0       0   50404       0       0]
 [    450       0       0     

In [14]:
model = GaussianNB()
t1_start = process_time() 
kfold = KFold(n_splits = 10, random_state = 1, shuffle = True)
results = cross_val_score(model, res_x, res_y, scoring = 'accuracy', cv = kfold)
predict = cross_val_predict(model, res_x , res_y,cv = 10)
print("Accuracy : ", results.mean())
print("Classification Report : \n", classification_report(predict, res_y))
print("Confusion Metrix : \n", confusion_matrix(predict, res_y))
t1_stop = process_time()
print("time:", t1_stop-t1_start)

C:\Users\deneysel\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\deneysel\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\deneysel\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\deneysel\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_

Accuracy :  0.6873328340648385
Classification Report : 
               precision    recall  f1-score   support

           0       0.44      0.99      0.61    901772
           1       0.87      0.92      0.89    852131
           2       0.99      0.87      0.92    646759
           3       1.00      0.87      0.93    533008
           4       1.00      0.81      0.89     62176
           5       0.78      0.00      0.01    981212
           6       0.90      0.18      0.31     39168

    accuracy                           0.69   4016226
   macro avg       0.85      0.66      0.65   4016226
weighted avg       0.79      0.69      0.62   4016226

Confusion Metrix : 
 [[891412   7829   2232      0      0     13    286]
 [ 66515 785176    118    196      0      0    126]
 [ 79286   6488 559602      0     40   1343      0]
 [ 37994  31546    111 463357      0      0      0]
 [ 11797      0     23      0  50356      0      0]
 [899427  74842   1675    168      0   4692    408]
 [ 29748   13

In [15]:
model = LogisticRegression()
t1_start = process_time() 
kfold = KFold(n_splits = 10, random_state = 1, shuffle = True)
results = cross_val_score(model, res_x, res_y, scoring = 'accuracy', cv = kfold)
predict = cross_val_predict(model, res_x , res_y,cv = 10)
print("Accuracy : ", results.mean())
print("Classification Report : \n", classification_report(predict, res_y))
print("Confusion Metrix : \n", confusion_matrix(predict, res_y))
t1_stop = process_time()
print("time:", t1_stop-t1_start)

C:\Users\deneysel\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\deneysel\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\deneysel\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\deneysel\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\deneysel\Anaconda3\lib\site-packages\sklearn\utils\validation

C:\Users\deneysel\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\deneysel\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\deneysel\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\deneysel\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\deneysel\Anaconda3\lib\site-packages\sklearn\utils\validation

Accuracy :  0.8783995221434088


C:\Users\deneysel\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Classification Report : 
               precision    recall  f1-score   support

           0       0.95      0.84      0.89   2266768
           1       0.84      0.94      0.88    811020
           2       0.99      0.88      0.93    639528
           3       0.64      0.99      0.78    298827
           4       0.00      0.00      0.00        83
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0

    accuracy                           0.88   4016226
   macro avg       0.49      0.52      0.50   4016226
weighted avg       0.91      0.88      0.89   4016226

Confusion Metrix : 
 [[1911769  139890    3901  167550   30894    4705    8059]
 [  46930  759663       0     108    4314       0       5]
 [  56022    6488  560424      55   15196    1343       0]
 [   1375    1239     205  296008       0       0       0]
 [     83       0       0       0       0       0       0]
 [      0       0       0       0       0       0       0]
 [